# HW2

## Part1 Beautiful Soup

### 載入套件

In [1]:
import requests
from bs4 import BeautifulSoup

### 取得網頁HMTL

In [14]:
res = requests.get("https://depart.moe.edu.tw/ed4500/cp.aspx?n=1AC243AF6EF5E5DD&s=EDC4A4E717ED32CF")
print(res.text)



<!DOCTYPE html>
<html id="FormHtml" xmlns="http://www.w3.org/1999/xhtml" lang="zh-Hant-tw">
<head id="Head1"><script>var CCMS_LanguageSN=1;</script><link rel="shortcut icon" href="https://www.edu.tw/favicon.ico" /><link rel="bookmark" href="https://www.edu.tw/favicon.ico" />

























<title>
	教育部統計處-統計表快速查詢
</title><meta charset="utf-8" /><meta http-equiv="X-UA-Compatible" content="IE=edge" /><meta name="viewport" content="width=device-width, initial-scale=1" /><meta name="DC.Title" content="教育部統計處全球資訊網" /><meta name="DC.Subject" content="教育部統計處" /><meta name="DC.Creator" content="教育部統計處" /><meta name="DC.Publisher" content="教育部統計處" /><meta name="DC.Date" content="2015/12/01" /><meta name="DC.Type" content="教育統計相關資料" /><meta name="DC.Identifier" content="2.16.886.101.20003.20005.20152" /><meta name="DC.Description" content="教育統計相關資料" /><meta name="DC.Contributor" content="" /><meta name="DC.Format" content="" /><meta name="DC.Relation" content="" /><meta name="DC.Sour

### 使用 Beautiful Soup 定位並取得資料

In [18]:
# 用 BeautifulSoup 方法解析 html
soup = BeautifulSoup(res.text, 'html.parser')
# 想取得資料的 XPath
# /html/body/form/div[1]/div[9]/div/div[3]/div[4]/div[3]/div/table/tbody/tr[2]/td/div/div[1]/ol[1]/li[1]/a
# /html/body/form/div[1]/div[9]/div/div[3]/div[4]/div[3]/div/table/tbody/tr[2]/td/div/div[2]/ol[1]/li[1]/a

# 用 BeautifulSoup select 相似路徑
tag_name = "tr td div div ol li a"
rows = soup.select(tag_name)
# 建立 list
link = []
name = []
# 取得網址與名字加入list中
for row in rows:
    link.append(row['href'])
    name.append(row.text)
print(link)
print(name)

['https://eds.moe.gov.tw/edust/webMain.aspx?sys=100&funid=eduout&funid2=B010100&cycle=4&outkind=1&outmode=8&defmk=1&outkind=1&fldlst=1111&codlst0=11111111111&codlst1=111&dfknd=1212', 'https://eds.moe.gov.tw/edust/webMain.aspx?sys=100&funid=eduout&funid2=B010200&cycle=4&outkind=1&outmode=8&defmk=1&outkind=1&fldlst=1111&codlst0=1&codlst1=1&dfknd=1212', 'https://eds.moe.gov.tw/edust/webMain.aspx?sys=100&funid=eduout&funid2=B020100&cycle=4&outkind=1&outmode=8&defmk=1&outkind=1&fldlst=11111111111&codlst0=1111111111111111111111111&codlst1=111&dfknd=1212', 'https://eds.moe.gov.tw/edust/webMain.aspx?sys=100&funid=eduout&funid2=B080100&cycle=4&outkind=1&outmode=8&defmk=1&outkind=1&fldlst=111&codlst0=1111111111111111111111111&codlst1=111&dfknd=1212', 'https://eds.moe.gov.tw/edust/webMain.aspx?sys=100&funid=eduout&funid2=B030100&cycle=4&outkind=1&outmode=8&defmk=1&outkind=1&fldlst=111111111111111111&codlst0=111&codlst1=111&dfknd=1212', 'https://eds.moe.gov.tw/edust/webMain.aspx?sys=100&funid=eduo

### 寫入CSV

In [21]:
# 想將 name 放 1st column
#   將 link 放 2nd column
# 將 name 、 link 用zip  放入 data
data = list(zip(name, link))

import csv

with open('統計處_統計表快速查詢.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    # 在first row 加入 Name 、 Link
    csv_writer.writerow(['Name', 'Link'])
    csv_writer.writerows(data)

## Part2 Selenium

### 載入套件

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_autoinstaller_fix
import time

import os
# 切換到這個檔案的位置
os.chdir(os.getcwd())

### 執行chrome webdriver

In [8]:
# 安裝chrome webdriver
chromedriver_autoinstaller_fix.install(path= os.getcwd())

# 讓webdriver開啟我們要的網站
driver = webdriver.Chrome(f'{os.getcwd()}/119/chromedriver.exe')
driver.get("https://depart.moe.edu.tw/ed4500/cp.aspx?n=1AC243AF6EF5E5DD&s=EDC4A4E717ED32CF")

# 點擊各級學校概況並按下
element = driver.find_element(By.CSS_SELECTOR,"#ContentPlaceHolder1_data_midlle_cp > table > tbody >\
                               tr:nth-child(2) > td > div > div:nth-child(2) > ol:nth-child(3) > li:nth-child(1) > a")
element.click()
time.sleep(2)
# 關閉一開始的webdriver
driver.close()
# 取得所有分頁
all_window_handles = driver.window_handles
# 將driver切換到新分頁
new_window_handle = all_window_handles[-1]
driver.switch_to.window(new_window_handle)

# 等待"sysdtcls" class出現代表加載完成
WebDriverWait(driver, 10).until(EC.presence_of_element_located(\
    (By.CSS_SELECTOR, "#grid > div.k-grid-header > div > table")))
# 取得查詢時間
time_path = "/html/body/form/div[2]/div[7]/span/span"
time = driver.find_element(By.XPATH, time_path)
time_text = time.text
download_time = time_text.encode('utf-8').decode('utf-8')
# 先建立一個陣列來放要存進csv的資料
table = [[download_time],["統計期","項目別","總計校數","公立校數","私立校數",\
          "總計學生數","公立學生數","私立學生數",\
          "總計教師數","公立教師數","私立教師數",\
          "總計上學期畢業生數","公立上學期畢業生數","私立上學期畢業生數"]]
# 取得各級學校_統計值
for i in range(1, 100):
    # 初始化
    data_row = []
    # 使用 XPath 選擇器來定位 tr 元素 並得到數量
    tr_selector = f'//*[@id="grid"]/div[2]/table/tbody/tr[{i}]'
    element_num = driver.find_element(By.XPATH, tr_selector)
    # 找到子元素
    child_elements = element_num.find_elements(By.TAG_NAME, "td")
    # 用enumerate方法將子元素數據存入 data_row
    for j, child_element in enumerate(child_elements):
        # 使用 XPath 選擇器來定位 td 元素
        td_selector = f'//*[@id="grid"]/div[2]/table/tbody/tr[{i}]/td[{j + 1}]'
        text = child_element.text
        # 將中文部分轉換為UTF-8格式
        utf8_text = text.encode('utf-8').decode('utf-8')
        data_row.append(utf8_text)
    table.append(data_row)

# 關閉webdriver
driver.close()

C:\Users\fdgg4\AppData\Local\Temp\ipykernel_1172\1822107379.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Python/Learning Analytics/HW2/119/chromedriver.exe')


### 輸出爬蟲完的資料

In [9]:
print(table)

[['112年11月05日(00:12)'], ['統計期', '項目別', '總計校數', '公立校數', '私立校數', '總計學生數', '公立學生數', '私立學生數', '總計教師數', '公立教師數', '私立教師數', '總計上學期畢業生數', '公立上學期畢業生數', '私立上學期畢業生數'], ['103學年', '總計', '11,078', '6,116', '4,962', '4,729,465', '2,985,206', '1,744,259', '303,078', '219,078', '84,000', '1,155,526', '752,705', '402,821'], ['103學年', '幼兒園', '6,468', '1,965', '4,503', '444,457', '134,154', '310,303', '45,341', '12,671', '32,670', '-', '-', '-'], ['103學年', '國小', '2,644', '2,607', '37', '1,252,762', '1,218,479', '34,283', '98,580', '96,742', '1,838', '242,831', '237,575', '5,256'], ['103學年', '國中', '738', '721', '17', '803,233', '708,894', '94,339', '52,154', '51,616', '538', '267,798', '238,449', '29,349'], ['103學年', '高級中等學校', '503', '292', '211', '818,866', '439,734', '379,132', '55,699', '36,424', '19,275', '277,047', '142,580', '134,467'], ['103學年', '大專校院', '159', '51', '108', '1,339,849', '434,655', '905,194', '49,357', '19,783', '29,574', '311,041', '109,548', '201,493'], ['103學年', '宗教研修學院', '2', '-',

### 寫入CSV

In [11]:
import csv

with open('各級學校_統計值.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(download_time)
    csv_writer.writerows(table)

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_autoinstaller_fix
import time
import os
# 切換到這個檔案的位置
os.chdir(os.getcwd())
# 安裝chrome webdriver
chromedriver_autoinstaller_fix.install(path= os.getcwd())

# 讓webdriver開啟我們要的網站
driver = webdriver.Chrome(f'{os.getcwd()}/119/chromedriver.exe')
driver.get("https://depart.moe.edu.tw/ed4500/cp.aspx?n=1AC243AF6EF5E5DD&s=EDC4A4E717ED32CF")

# 點擊各級學校概況並按下
element = driver.find_element(By.CSS_SELECTOR,"#ContentPlaceHolder1_data_midlle_cp > table > tbody >\
                               tr:nth-child(2) > td > div > div:nth-child(2) > ol:nth-child(3) > li:nth-child(1) > a")
element.click()
time.sleep(2)
# 關閉一開始的webdriver
driver.close()
# 取得所有分頁
all_window_handles = driver.window_handles
# 將driver切換到新分頁
new_window_handle = all_window_handles[-1]
driver.switch_to.window(new_window_handle)

# 等待"sysdtcls" class出現代表加載完成
WebDriverWait(driver, 10).until(EC.presence_of_element_located(\
    (By.CSS_SELECTOR, "#grid > div.k-grid-header > div > table")))
# 取得查詢時間
time_path = "/html/body/form/div[2]/div[7]/span/span"
time = driver.find_element(By.XPATH, time_path)
time_text = time.text
download_time = time_text.encode('utf-8').decode('utf-8')
# 先建立一個陣列來放要存進csv的資料
table = [[download_time],["統計期","項目別","總計校數","公立校數","私立校數",\
          "總計學生數","公立學生數","私立學生數",\
          "總計教師數","公立教師數","私立教師數",\
          "總計上學期畢業生數","公立上學期畢業生數","私立上學期畢業生數"]]
# 取得各級學校_統計值
for i in range(1, 100):
    # 初始化
    data_row = []
    # 使用 XPath 選擇器來定位 tr 元素 並得到數量
    tr_selector = f'//*[@id="grid"]/div[2]/table/tbody/tr[{i}]'
    element_num = driver.find_element(By.XPATH, tr_selector)
    # 找到子元素
    child_elements = element_num.find_elements(By.TAG_NAME, "td")
    # 用enumerate方法將子元素數據存入 data_row
    for j, child_element in enumerate(child_elements):
        # 使用 XPath 選擇器來定位 td 元素
        td_selector = f'//*[@id="grid"]/div[2]/table/tbody/tr[{i}]/td[{j + 1}]'
        text = child_element.text
        # 將中文部分轉換為UTF-8格式
        utf8_text = text.encode('utf-8').decode('utf-8')
        data_row.append(utf8_text)
    table.append(data_row)

# 關閉webdriver
driver.close()
import csv

with open('各級學校_統計值test.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(download_time)
    csv_writer.writerows(table)

c:\Python\Learning Analytics\HW2


C:\Users\fdgg4\AppData\Local\Temp\ipykernel_1568\592909133.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(f'{os.getcwd()}/119/chromedriver.exe')
